In [60]:
import cv2
import numpy as np

# Görüntüyü açma
image_path = "gul.jpeg"
image = cv2.imread(image_path)

# Boyut ayarlama
scale_percent = 50
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized_image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

# Görüntüyü HLS renk uzayına dönüştür
hls_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2HLS)

# Kırmızı renk tonlarına karşılık gelen bir maske oluşturun
lower_red1 = np.array([0, 10,10 ])
upper_red1 = np.array([10, 255, 255])
mask1 = cv2.inRange(hls_image, lower_red1, upper_red1)

lower_red2 = np.array([100, 30, 30])
upper_red2 = np.array([255, 255, 255])
mask2 = cv2.inRange(hls_image, lower_red2, upper_red2)

# Maskeleri birleştirin
mask = cv2.bitwise_or(mask1, mask2)

# Maskeyi pürüzsüzleştirmek için morfolojik işlemler
kernel = np.ones((0, 0), np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)


# Daha küçük ve büyük maskeleri kaldırmak için ek morfolojik işlemler
mask = cv2.erode(mask, kernel, iterations=1)
mask = cv2.dilate(mask, kernel, iterations=2)

# Gülün rengini mor yapmak için HLS değerlerini değiştirin
hls_image[mask > 0, 0] = 270 // 2  # HUE değerini mor yapmak için 270 dereceye ayarla (OpenCV H değeri [0, 180] aralığında)
gulu_mor_yap = cv2.cvtColor(hls_image, cv2.COLOR_HLS2BGR)

# Arka planı siyah yapın
background_black = resized_image.copy()
background_black[mask == 0] = [0, 0, 0]

# Gülün mor olan versiyonunu siyah arka plan ile birleştirin
mor_gul = gulu_mor_yap.copy()
mor_gul[mask == 0] = [0, 0, 0]

# Sonuçları göster
cv2.imshow('Orijinal Görsel', resized_image)
cv2.imshow('Arka Plan Siyah', background_black)
cv2.imshow('Mor Gül', mor_gul)

# Görüntüleri kaydedin
cv2.imwrite(r'C:\Users\batuhan\Desktop\staj\siyah_arka_plan.png', background_black)
cv2.imwrite(r'C:\Users\batuhan\Desktop\staj\mor_gul.png', mor_gul)

# Çıkmak için bir tuşa basın
cv2.waitKey(0)
cv2.destroyAllWindows()
